This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-run-anywhere/edge-architecture-publish-linear-regression-houseprice-model).

## Run Anywhere With Acceleration Tutorial: Aloha Model

Wallaroo supports deploying models with optimizations that increase the inference speed and performance.  These optimizations are set during model upload, and are carried through to model deployment and model edge deployment.

### Supported Optimizations

The following optimizations are supported:

| Accelerator | Description |
|---|---|
| `None` | The default acceleration, used for all scenarios and architectures. |
| `Aio` | Compatible only with the `ARM` architecture. |
| `Jetson` | compatible only with the `ARM` architecture.
| `CUDA` | Compatible with either `ARM` or `X86/X64` architectures. |


## Goal

Demonstrate uploading an Aloha model with the `Jetson`, then publishing the same model for edge deployment with the `Jetson` optimization inherited from the model.

### Resources

This tutorial provides the following:

* Models:
  * `models/alohacnnlstm.zip`:  An open source model based on the [Aloha CNN LSTM model](https://www.researchgate.net/publication/348920204_Using_Auxiliary_Inputs_in_Deep_Learning_Models_for_Detecting_DGA-based_Domain_Names) for classifying Domain names as being either legitimate or being used for nefarious purposes such as malware distribution.  

### Prerequisites

* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
  * `json`: Used for format input data for inference requests.


## Steps

* Upload the model with the targeted optimization left as `None` by default.
* Create the pipeline add the model as a model step.
* Deploy the model with deployment configuration and show the acceleration setting inherits the model's optimization.
* Publish the pipeline an OCI registry and show the publish pipeline deployment configuration inherit's the model's optimization.

### Import Libraries

The first step will be to import our libraries, and set variables used through this tutorial.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError

# to display dataframe tables
from IPython.display import display
# used to display dataframe information without truncating
import pandas as pd
pd.set_option('display.max_colwidth', None)
import pyarrow as pa

In [2]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

### Connect to the Wallaroo Instance

The next step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [4]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace.

Workspace, pipeline, and model names should be unique to each user, so we'll add in a randomly generated suffix so multiple people can run this tutorial in a Wallaroo instance without effecting each other.

In [5]:
workspace_name = 'optimization-aloha-demonstration'

workspace = get_workspace(workspace_name, wl)

wl.set_current_workspace(workspace)

{'name': 'optimization-aloha-demonstration', 'id': 2900937, 'archived': False, 'created_by': 'b4a9aa3d-83fc-407a-b4eb-37796e96f1ef', 'created_at': '2024-04-01T21:20:03.337377+00:00', 'models': [], 'pipelines': []}

### Set Model Optimization at Model Upload

For our example, we will upload the model.  The file name is `./models/alohacnnlstm.zip` and the model will be called `aloha`.

Models are uploaded to Wallaroo via the [`wallaroo.client.upload_model`](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-reference-guide/client/#Client.upload_model) method which takes the following arguments:

| Parameter | Type | Description |
|---|---|---|
| **path** | *String* (*Required*) | The file path to the model. |
| **framework** | *wallaroo.framework.Framework* (*Required*) | The model's framework.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for supported model frameworks. |
| **input_schema** | *pyarrow.lib.Schema* (*Optional*)  | The model's input schema.  **Only required for non-Native Wallaroo frameworks.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for more details. |
| **output_schema** | *pyarrow.lib.Schema* (*Optional*)  | The model's output schema.  **Only required for non-Native Wallaroo frameworks.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for more details. |
| **convert_wait** | *bool* (*Optional*)  | Whether to wait in the SDK session to complete the auto-packaging process for non-native Wallaroo frameworks. |
| **arch** | *wallaroo.engine_config.Architecture* (*Optional*)  | The targeted architecture for the model.  Options are <ul><li>`X86` (*Default*)</li><li>`ARM`</li></ul> |
| **accel** |  *wallaroo.engine_config.Acceleration* (*Optional*)  | The targeted optimization for the model.  Options are <ul><li>`None`: The default acceleration, used for all scenarios and architectures.</li><li>`Aio`:Compatible only with the `ARM` architecture.</li><li>`Jetson`: Compatible only with the `ARM` architecture.</li><li>`CUDA`: Compatible with either `ARM` or `X86/X64` architectures.</li></ul> |

We upload the model and set the `accel` to `wallaroo.engine_config.Acceleration.Jetson` with the Architecture set to `ARM`.

In [6]:

model_name = 'aloha'
model_file_name = './models/alohacnnlstm.zip'

from wallaroo.framework import Framework
from wallaroo.engine_config import Architecture, Acceleration

model = wl.upload_model(model_name, 
                        model_file_name,
                        framework=Framework.TENSORFLOW,
                        arch=Architecture.ARM,
                        accel=Acceleration.Jetson,
                        )

#### Display Model Details

Once the model is uploaded, we view the model details to verify the `accel` setting it set to `Jetson`.

In [7]:
model

Name,aloha
Version,43d1d119-1aea-43c3-8d2d-c72394ee0d06
File Name,alohacnnlstm.zip
SHA,d71d9ffc61aaac58c2b1ed70a2db13d1416fb9d3f5b891e5e4e2e97180fe22f8
Status,ready
Image Path,None
Architecture,arm
Acceleration,jetson
Updated At,2024-01-Apr 21:20:04


#### Create the Pipeline

With the model uploaded, we build our pipeline and add the Aloha model as a pipeline step.

In [19]:
pipeline_name = 'aloha-pipeline'

aloha_pipeline = wl.build_pipeline(pipeline_name)

_ = aloha_pipeline.add_model_step(model)

### Set Optimization for Pipeline Publish

Publishing the pipeline uses the pipeline `wallaroo.pipeline.Pipeline.publish()` command.  This requires that the Wallaroo Ops instance have [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) enabled.

The model's optimization setting can be **overridden** during the pipeline publish stage.  This allows edge deployments with different optimizations and accelerators to be used depending on the deployment environment.

In this example, first we will publish the pipeline with a deployment configuration that does not change the `accel` or `arch` setting.  The `Engine Config` for the pipeline publish will inherit these settings from the model.

Then we will deploy the pipeline again and set the accelerator as `wallaroo.engine_config.Acceleration.CUDA` to show how to override the engine deployment inheritance.

The following publishes the pipeline to the OCI registry and displays the container details.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/).

In [16]:
from wallaroo.deployment_config import DeploymentConfigBuilder

deploy_config_none = (wallaroo.DeploymentConfigBuilder()
                     .replica_count(1)
                     .cpus(1)
                     .memory("1Gi")
                     .build()
                    )

aloha_pipeline.publish(deployment_config=deploy_config_none)

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing..................... Published.


ID,73
Pipeline Name,aloha-pipeline
Pipeline Version,15af3572-2cc2-42c7-8cbe-ccf6e3605aa0
Status,Published
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini-jetson:v2024.1.0-main-4833
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/aloha-pipeline:15af3572-2cc2-42c7-8cbe-ccf6e3605aa0
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/aloha-pipeline
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:4b424872d763bd285ddef9785b0f6d748eec666b59c3cac9d9e36d387a9526e5
Helm Chart Version,0.0.1-15af3572-2cc2-42c7-8cbe-ccf6e3605aa0
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}, 'accel': 'jetson', 'arch': 'arm', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': {}}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}"
User Images,[]


We publish the pipeline again, this time changing the `accel` to `wallaroo.engine_config.Acceleration.CUDA`.  Note the `Engine Config` is updated to match the change in the optimization.

In [18]:
from wallaroo.deployment_config import DeploymentConfigBuilder

deploy_config_cuda = (wallaroo.DeploymentConfigBuilder()
                     .replica_count(1)
                     .cpus(1)
                     .memory("1Gi")
                     .arch(wallaroo.engine_config.Architecture.ARM)
                     .accel(wallaroo.engine_config.Acceleration.CUDA)
                     .build()
                    )

aloha_pipeline.publish(deployment_config=deploy_config_cuda)

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing................. Published.


ID,75
Pipeline Name,aloha-pipeline
Pipeline Version,4417f9ea-16da-4a62-8bf3-3154367328aa
Status,Published
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini-cuda-aarch64:v2024.1.0-main-4833
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/aloha-pipeline:4417f9ea-16da-4a62-8bf3-3154367328aa
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/aloha-pipeline
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:7058caefab3c46f13bc2c464b969c502a22de2e5ab18c6e4fddd4c0df215bb6b
Helm Chart Version,0.0.1-4417f9ea-16da-4a62-8bf3-3154367328aa
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}, 'accel': 'cuda', 'arch': 'arm', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': {}}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}"
User Images,[]
